이제까지 X와 Y가 모두 주어진 훈련상황을 가정했습니다. 이번에는 X만 주어진 상태에서 $\hat{Y}$을 추론하는 방법에 관해 서술하겠습니다. 이러한 과정을 추론 또는 $탐색^{search}$라고 부릅니다. 결국 우리가 원하는 것은 단어들 사이에서 최고의 확률을 갖는 $경로^{path}$를 찾는 것입니다.

### Sampling

먼저 떠올릴 수 있는 가장 정확한 방법은 각 time-step별 $\hat{y_t}$을 고릴때 마지막 softmax 계층에서의 확률 분포대로 샘플링하는 것입니다. 그리고 다음 time-step에서 그 선택 $\hat{y_t}$을 기반으로 다음 $\hat{y_{t+1}}$을 도 다시 샘플링하여 최종적으로 EOS가 나올때까지 샘플링을 반복합니다. 이렇게 하면 우리가 원하는 분포에 가장 가까운 형태의 번역이 완성될 것입니다. 

하지만 이러한 방식은 같은 입력에 대해 매번 다른 출력 결과물을 만들어 낼 수 있습니다. 따라서 우리가 원하는 형태의 결과물은 아닙니다.

<br></br>
$$
\hat{y_t} \sim P(y_t|X,\hat{y_{<t}};\theta)
$$
<br></br>

### Greedy Search Algorithm

탐욕 탐색 알고리즘을 기반으로 탐색을 구현해보겠습니다. 즉, 소프트맥스 계층에서 가장 확률값이 큰 인덱스를 뽑아 해당 time-step의 $\hat{y_t}$으로 사용하는 것입니다. 이를 수식으로 나타내면 다음과 같습니다.

<br></br>
$$
\hat{y_t} = argmax_{y \in Y} P(y_t|X,\hat{y_{<t}};\theta)
$$
<br></br>

<br></br>
![](./images/10-6-2-greedy.jpg)
<br></br>

### 파이토치 코드

다음은 Greedy Search Algorithm을 위한 코드입니다. 인코더가 동작하는 부분까지는 완전히 똑같습니다. 다만, 이후 추론을 위한 부분은 teacher forcing을 사용했던 훈련 방식과 달리, 실제 이전 time-step의 출력을 현재 time-step의 입력으로 사용합니다.

In [1]:
def search(self, src, is_greedy = True, max_length = 255):
    mask, x_length = None, None
    
    if isinstance(src, tuple):
        x, x_length = src
        mask = self.generate_mask(x, x_length)
        
    else:
        x = src
        
    batch_size = x.size(0)
    
    emb_src = self.emb_src(x)
    h_src, h_0_tgt = self.encoder((emb_src, x_length))
    h_0_tgt, c_0_tgt = h_0_tgt
    h_0_tgt = h_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size).transpose(0,1).contiguous()
    
    c_0_tgt = c_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size).transpose(0,1).contiguous()
    
    h_0_tgt = (h_0_tgt, c_0_tgt)
    
    ## Fill a vector, which has "batch_size" dimension with BOS value.
    y = x.new(batch_size, 1).zero_() + data_loader.BOS
    is_undone = x.new_ones(batch_size, 1).float()
    decoder_hidden = h_0_tgt
    h_t_tilde, y_hats, indice = None, [], []
    
    ## Repeat a loop while sum of "is_undone" flag is bigger than 0
    ## or current time-step is smaller than maximum length
    
    while is_undone.sum() > 0 and len(indice) < max_length:
        
        ## Unlike training procedure, take the last time-step's output during the inference.
        ## |emb_t| = (batch_size, 1, word_vec_dim)
        emb_t = self.emb_dec(y)
        
        decoder_output, decoder_hidden = self.decoder(emb_t, 
                                                      h_t_tilde, 
                                                      decoder_hidden)
        
        context_vector = self.attn(h_src, decoder_output, mask)
        h_t_tilde = self.tanh(self.concat(torch.cat([decoder_output,
                                                     context_vector], dim = -1)))
        
        ## |y_hat| = (batch_size, 1, output_size)
        y_hat = self.generator(h_t_tilde)
        
        y_hats += [y_hat]
        
        if is_greedy:
            y = torch.topk(y_hat, 1, dim = -1)[1].squeeze(-1)
            
        else:
            ## Take a random sampling baed on the multinoulli distribution
            y = torch.multinomial(y_hat.exp().view(batch_size,-1), 1)
            
        ## Put PAD if the sample is done
        
        ## |y| = (batch_size, 1)
        ## |is_undone| = (batch_size, 1)
        y = y.masked_fill_((1. - is_undone).byte(), data_loader.PAD)
        is_undone = is_undone * torch.ne(y, data_loader.EOS).float()
        
        indice += [y]
    
    ## |y_hats| = (batch_size, length, output_size)
    ## |indice| = (batch_size, length)
    y_hats = torch.cat(y_hats, dim = 1)
    indice = torch.cat(indice, dim = -1)
    
    return y_hats, indice

가끔 너무 어렵거나 훈련 데이터에서 볼 수 없었던 형태의 문장이 인코딩되어 들어오거나, 훈련 데이터가 적어서 디코더가 잘 훈련되어 있지 않으면, 같은 단어를 반복하며 끝이 없는 문장을 뱉어내는 현상이 발생할 수 있습니다. 즉, EOS가 나오지 않는 상황이 발생할 수 있습니다.

<br></br>

|정상|비정상|
|---|----|
|나는 학교에 갑니다.|나는 학교에 학교에 학교에 학교에 학교에 ...|

<br></br>

따라서 우리는 앞의 함수 입력에서 볼 수 있듯이, 최대 가능 문장 길이를 정해주어 끝이 없는 문장이 나오는 경우에 대비합니다.

### Beam Search

Greedy Search Algorithm은 매우 쉽고 간편하지만 최적의 해를 보장하지는 않습니다. 따라서 최적의 해에 더 가까워지기 위해 k개의 후보를 더 추적합니다. 이때 k를 **beam**이라고 부릅니다.

<br></br>
![](./images/10-6-4-beam.jpg)
<br></br>

현재 time-step에서 최고 누적 확률을 가진 인덱스의 단어 k개를 뽑은뒤 다음 time-step에 대해 k번 추론을 수행합니다. 그리고 총 k|V|개의 `softmax` 결과값 중 다시 최고 누적 확률 k개를 뽑아 다음 time-step으로 넘깁니다. 여기서 중요한 점은 다음과 같은 2가지입니다.

1. 누적 확률을 사용하여 최고 순위 k개를 뽑습니다. 이때 보통 로그 확률을 사용하므로 현재 time-step까지의 로그 확률에 대한 합을 추적하고 있어야 합니다.
2. 이전 time-step에서 뽑힌 k개에 대해 계산한 현재 time-step의 모든 결과물 중에서 최고 누적확률 k개를 다시 뽑습니다. 이전 time-step의 k개에 대해 각각 k개를 뽑는게 아닙니다.

Beam Search를 사용하면 더 넓은 경로에 대해 탐색하므로 더 나은 성능을 보장합니다. 하지만 빔의 크기만큼 연산을 더 수행하므로 속도 저하가 일어납니다.

다음은 빔서치의 성능 향상에 대한 실험 결과입니다.

<br></br>

|알고리즘|Valid set NLL (BLEU)|Test set NLL (BLEU)|
|------|--------------------|-------------------|
|샘플링|22.98 (15.64)|26.25 (16.76)|
|Greedy|27.88 (15.50)|26.49 (16.66)|
|Beam (k=5)|20.18 (17.03)|22.81 (18.56)|
|Beam (k=10)|19.92 (17.13)|22.44 (18.59)|

<br></br>

#### 구현 관점에서 바라보기

앞에서 살펴본 그림은 하나의 샘플에 대해 추론을 수행하기 위한 빔서치를 나타낸 것입니다. 여기서 빔의 크기는 k = 3입니다. 보통 빔서치는 EOS가 k만큼 나올떄까지 수행됩니다. 즉 앞으로 우리가 다룰 예제에서는 3개 이상의 EOS가 출현하면 빔서치는 종료됩니다.

앞에서 본 그림에서는 마지막 직전의 time-step에서 1개의 EOS가 발생했고, EOS에서 다시 이어지는 추론 대신 다른 2개의 추론중 최고 누적 확률 3개를 다시 선택하는 것을 볼 수 있습니다. 우리는 EOS 이후의 해당 빔의 누적 확률값을 0으로 주어 이것을 쉽게 구현할 수 있습니다.

<br></br>
$$
\text{Given}\ X = x_1, x_2, \dots, x_n, \\
\text{we need to find}\ b^* \\
\text{where}\ B_t = \{ b_t^1, b_t^2, \dots, b_t^K \},b_t^i = \{\hat{y_1^i}, \dots, \hat{y_t^i}\}
$$ 
<br></br>

<br></br>
$$
\hat{y_i^k} = argmax_{y \in Y}^k \{log P_\theta(y_t|X,b_{t-1}^1 + log P_\theta(b_{t-1}^K|X), \dots, log P_\theta(y_t|X,b_{t-1}^1) + log P_\theta(b_{t-1}^K|X) \}
$$ 
<br></br>

<br></br>
$$
b_t^k = \{ \hat{y_1^k}, \dots, \hat{y_{t-1}^k} \} + \{\hat{y_t^k}\} \\
= \{ \hat{y_1^k}, \dots, \hat{y_t^k}\} \\
= argmax_{b \in B_t}^k \{log P_\theta(y_t|X,b_{t-1}^1 + log P_\theta(b_{t-1}^K|X), \dots, log P_\theta(y_t|X,b_{t-1}^1) + log P_\theta(b_{t-1}^K|X)\} + \{ \hat{y_t^k} \}
$$ 
<br></br>

<br></br>
$$
log P_\theta(b_t^i|X) = log P_\theta(y_t = \hat{y_t^i}|X) + log P_\theta(b_t^i|X) \\
= \sum_{t=1}^t log P_\theta (y_t = \hat{y_t^i}|X,\hat{y_{<t}^i})
$$
<br></br>

예를 들어 |V| = 30,000이고, k = 3이라고 가정하겠습니다. 그럼 소프트맥스 계층의 유닛 개수는 30,000개입니다. 따라서 한 time-step의 예측 결과의 개수는 |V| x k = 90,000이 됩니다. 이 9만개의 소프트맥스 계층 유닛 (vocabulary의 각 단어)의 각각의 확률값에, 이전 time-step에서 선택된 빔의 누적 확률값을 더해 최종 누적 확률값 9만개를 얻습니다. 여기서 총 90,000개의 누적 확률값중에서 최고의 k개를 뽑아 다음 time-step의 예측을 위한 디코더의 입력으로 사용합니다. 이때 반복하는 k번의 작업을 for반복문으로 해결할 수도 있지만, 미니배치 크기가 마치 k인 것처럼 미니 배치를 구성해서 seq2seq에 넣어주면 됩니다. 이것은 한개의 문장이 주어졌을때의 시나리오이고, 미니배치로 문장이 주어진다면 $batch size x |V| x k$가 될 것입니다.

<br></br>
![](./images/10-6-4-beamsearch.jpg)
<br></br>

<br></br>
![](./images/10-6-4-minibatch.jpg)
<br></br>

seq2seq은 미니배치를 입력으로 받고 미니배치를 출력으로 반환합니다. 따라서 상황에 따라 필요한 입력들을 모아서 미니배치를 만들고, 그에 해달하는 출력을 비니배치로 받아 입력에 대해 필요한 출력을 나누어 가지면 됩니다. 아까 1개의 문장에 대해 설명할대, k번 반복하는 작업에 대해 for문을 사용하는 것 대신, 마치 미니배치 크기가 k개인 것처럼 미니배치를 만들어 병렬 연산을 통해 구현하는 것을 이야기 하였습니다. 

m개의 문장이 주어지는 미니배치 상황에서도 똑같습니다. 문장별로 k번 반복하는 작업이 있으므로, 결국 최종 임시 미니 배치 크기는 m x k가 될 것입니다. 우리는 매 time-step마다 각 m개의 문장별로 최고 누적 확률 k개의 입력을 뽑아내서 m x k 크기의 임시 미니배치를 구성하여 seq2seq에 입력으로 넣습니다. seq2seq가 m x k크기의 임시 미니배치를 출력으로 반환하면, 다시 m개의 문장별로 k개씩 확률 분포를 뽑아서, 최고 누적 확률을 계산하여 다음 time-step의 입력이 될것을 정하면 됩니다.

#### 빔서치 수행 결과 비교

경험상 빔서치를 제대로 구현하면 BLEU 성능이 2가량 올라가는 것을 볼 수 있습니다. 다음은 빔의 크기 k = 1인 단순한 Greedy Search Algorithm과 k = 5인 빔서치의 결과를 비교한 예제입니다. 대체적으로 표현이 더 풍부해지는 것을 알 수 있습니다.

<br></br>

|번호|빔 크기 k = 1|빔 크기 k = 5|
|---|------------|-----------|
|1|너 스스로 말하고, 나를 위해 말하지마|당신 자신을 위해 말하세요, 당신을 나를 위해 말하지 않아요.|
|2|걱정마. 난 그것에 익숙해|걱정마. 난 그것에 익숙해져 있어.|
|3|너는 내 상사에게 말해야 한다.|너는 나의 상사에게 말해야 한다.|
|4|그녀는 그와 결혼하지 않기로 결심한 것을 느꼈다.|그녀는 그와 결혼하지 않기로 결심한 그녀의 결심을 느꼈다.|
|5|배달 채널을 삭제할 수 없습니다.|배달 채널이 삭제되지 않았습니다.|
|6|하지만 여러모로 나는 행복한 사람이 나를 화나게 하지 않는다.|하지만 많은 면에서 나는 행복한 사람이 나를 화나게 하지 않는다.|
|7|그 점원의 불의의 정중함은 나를 기분 좋게 만들었다.|그 점원의 예상치 못한 공손함이 나를 기분 좋게 만들었다.|
|8|내가 너의 조종사를 보고 있는 것을 신경쓰지않기를 바란다.|내가 너의 조종사를 감시하는 것을 신경쓰지 않았으면 좋겠어|

<br></br>

#### 길이 패널티

앞의 탐색 알고리즘을 코딩하여 직접 실행시키면 문제점이 있습니다. 현재 time-step까지의 확률을 모두 곱하므로 문장이 길어질수록 확률이 낮아진다는 점입니다. 따라서 짧은 문장일수록 더 높은 점수를 획득하는 경향이 있습니다. 이러한 현상을 피하기 위해 예측한 문장의 길이에 따른 페널티를 주어 짧은 문장이 긴 문장을 제치고 선택되는 것을 방지합니다.

길이 패널티의 수식은 다음과 같습니다. 이를 위해서는 추가적인 하이퍼 파라미터를 갖습니다. 보통 $\alpha = 1.2, \beta = 5$의 값을 갖습니다.

<br></br>
$$
log \tilde{P}(\hat{Y}|X) = log P(\hat{Y}|X) x penalty \\
penalty = \frac{(1+length)^\alpha}{(1+\beta)^\alpha}
$$
<br></br>

### 파이토치 예제

#### SingleBeamSearchSpace 클래스

지금부터 살펴볼 코드는 하나의 문장에 대한 빔서치를 수행하기 위한 클래스입니다. 따라서 해당 클래스의 객체는 입력으로 주어진 미니배치의 크기 (문장의 개수)만큼 선언됩니다.

##### 클래스 선언 및 초기화

In [2]:
from operator import itemgetter

import torch
import torch.nn
import data_loader

LENGTH_PENALTY = 1.2
MIN_LENGTH = 5

class SingleBeamSearchSpace():
    
    def __init__(self,
                hidden,
                h_t_tilde = None,
                beam_size = 5,
                max_length = 255):
        
        self.beam_size = beam_size
        self.max_length = max_length
        
        super(SingleBeamSearchSpace, self).__init__()
        
        ## To put data to the same device
        self.device = hidden[0].device
        
        ## Inferred word index for each time-step.
        ## For now, initialized with initial time-step
        self.word_indice = [torch.LongTensor(beam_size).zero_().to(self.device) + data_loader.BOS]
        
        ## Index origin of current beam
        self.prev_beam_indice = [torch.LongTensor(beam_size).zero_().to(self.device) - 1]

        ## Cumulative log-prob for each beam
        self.cumulative_probs = [torch.FloatTensor([.0] + [-float("inf")] * (beam_size - 1)).to(self.device)]
        
        ## 1 if it is done else 0
        self.masks = [torch.ByteTensor(beam_size).zero_().to(self.device)]
        
        ## We don't need to remember every time-step of hidden states:
        ## prev_hidden, prev_cell, prev_h_t_tilde
        ## we only need the last one
        
        ## |hidden[0]| = (n_layers, 1, hidden_size)
        ## |prev_hidden| = (n_layers, beam_size, hidden_size)
        self.prev_hidden = torch.cat([hidden[0]] * beam_size, dim = 1)
        
        ## |prev_cell| (n_layers, beam_size, hidden_size)
        self.prev_cell = torch.cat([hidden[1]] * beam_size, dim = 1)
        
        ## |h_t_tilde| = (batch_size = 1, 1, hidden_size)
        ## |prev_h_t_tilde| = (batch_size, 1, hidden_size)
        self.prev_h_t_tilde = torch.cat([h_t_tilde] * beam_size, dim = 0) if h_t_tilde is not None else None
        
        self.current_time_step = 0
        self.done_cnt = 0

ModuleNotFoundError: No module named 'data_loader'

##### 길이 패널티 구현하기

In [3]:
def get_length_penalty(self,
                       length,
                       alpha = LENGTH_PENALTY,
                       min_length = MIN_LENGTH):
    
    ## Calculate length -penalty
    ## Because shorter sentence usually have bigger probability.
    ## Thus we need to put penalty for shorter one
    p = (1 + length) ** alpha / (1 + min_length) ** alpha
    
    return p

NameError: name 'LENGTH_PENALTY' is not defined

##### 디코딩 작업 종료 체크

EOS가 한개씩 나타날때마다 self.done_cnt가 증가하므로, 빔의 크기 k보다 self.done_cnt가 크면 디코딩 작업이 종료됩니다.

In [4]:
def is_done(Self):
    if self.done_cnt >= self.beam_size:
        return 1
    return 0

##### 가짜 미니배치의 일부 만들기

매 time-step마다 seq2seq에게 1개의 문장에 대한 k개의 입력이 주어져야 합니다. 배치크기가 m일때는 m개의 문장에 대해 각각 k개의 입력이 될 것입니다. 최종 미니배치의 크기는 m x k가 될것이므로 그 일부인 k개를 반환하는 함수입니다.

In [5]:
def get_batch(self):
    ## |y_hat| = (beam_size, 1)
    y_hat = self.word_indice[-1].unsqueeze(-1)
    
    ## |hidden| = (n_layers, beam_size, hidden_size)
    hidden = (self.prev_hidden, self.prev_cell)
    
    ## |h_t_tilde| = (beam_size, 1, hidden_size) or None
    h_t_tilde = self.prev_h_t_tilde
    
    return y_hat, hidden, h_t_tilde

##### 최고 누적 확률 k개 고르기

매 time-step마다 k개의 결과값을 받아 이전 time-step으로부터의 빔서치 정보를 이용하여 누적확률을 계산하고, 최고 누적확률 k개를 뽑는 작업을 수행하는 함수입니다. self.mask의 값의 따라 이전에 EOS가 출현한 경우에는 직전 누적확률을 0으로 만들어 계산하는 것을 볼 수 있습니다.

In [ ]:
def collect_result(self, y_hat, hidden, h_t_tilde):
    ## |y_hat| = (beam_size, 1, output_size)
    ## |hidden| = (n_layers, beam_size, hidden_size)
    ## |h_t_tilde| = (beam_size, 1, hidden_size)  
    output_size = y_hat.size(-1)
    
    self.current_time_step += 1
    
    ##